In [2]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup
import re

# Data collection 1 single page 

In [3]:
url='https://www.flipkart.com/search?q=smart+watches&sid=ajy%2Cbuh&as=on&as-show=on&otracker=AS_QueryStore_HistoryAutoSuggest_1_5_na_na_na&otracker1=AS_QueryStore_HistoryAutoSuggest_1_5_na_na_na&as-pos=1&as-type=HISTORY&suggestionId=smart+watches%7CSmart+Watches&requestId=fe84e6ac-d178-47e4-a3ba-afce1aa1b5b9&as-backfill=on'
page=requests.get(url)
page

<Response [529]>

In [4]:
url

'https://www.flipkart.com/search?q=smart+watches&sid=ajy%2Cbuh&as=on&as-show=on&otracker=AS_QueryStore_HistoryAutoSuggest_1_5_na_na_na&otracker1=AS_QueryStore_HistoryAutoSuggest_1_5_na_na_na&as-pos=1&as-type=HISTORY&suggestionId=smart+watches%7CSmart+Watches&requestId=fe84e6ac-d178-47e4-a3ba-afce1aa1b5b9&as-backfill=on'

In [5]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,/;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    # 'Accept-Encoding': 'gzip, deflate, br',
    'DNT': '1',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
}

In [6]:
page=requests.get(url,headers=headers)
page

<Response [200]>

In [7]:
soup=BeautifulSoup(page.text)
soup

<!DOCTYPE html>
<html lang="en"><head><link href="https://rukminim2.flixcart.com" rel="preconnect"/><link href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app_modules.chunk.c48a12.css" rel="stylesheet"/><link href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app.chunk.066267.css" rel="stylesheet"/><meta content="text/html; charset=utf-8" http-equiv="Content-type"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="102988293558" property="fb:page_id"/><meta content="658873552,624500995,100000233612389" property="fb:admins"/><link href="https://static-assets-web.flixcart.com/www/promos/new/20150528-140547-favicon-retina.ico" rel="shortcut icon"/><link href="/osdd.xml?v=2" rel="search" type="application/opensearchdescription+xml"/><meta content="website" property="og:type"/><meta content="Flipkart.com" name="og_site_name" property="og:site_name"/><link href="/apple-touch-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/><l

In [8]:
def text_from_tag(elem):
    # prefer title attribute (Flipkart uses title), else text
    if hasattr(elem, 'get'):
        return (elem.get('title') or elem.get_text(separator=' ')).strip()
    return str(elem)

rx_inch = re.compile(r'\((\d+(?:\.\d+)?)\)|(\d+(?:\.\d+)?)(?=\s*(?:["\'’]{1,2}|inch\b|in\b))', re.I)
rx_mm   = re.compile(r'(\d+(?:\.\d+)?)\s*mm\b', re.I)



price=soup.find_all('div',class_="Nx9bqj")
prices=[]
x=0
for i in price:
    prices.append(i.text)
    x=x+1
    if x==40:
        break
mrp=soup.find_all('div',class_="yRaY8j")
MRP=[]
y=0
for i in mrp:
    MRP.append(i.text)
    y=y+1
    if y==40:
        break
off=soup.find_all('div',class_="UkUFwK")
offer=[]
z=0
for i in off:
    offer.append(i.text.split('%')[0])
    z=z+1
    if z==40:
        break

# --------------------------

clr=soup.find_all('div',class_="Br9IW+")
color=[]
for i in clr:
    color.append(i.text.split(',')[0])

belt_size=[]
for i in clr:
    belt_size.append(i.text.split(',')[1])

# ---------------------------

data=soup.find_all('a',class_="WKTcLC")
brand=[]
for i in data:
    brand.append(i.text.split()[0])

model=[]
for i in data:
    model.append(i.text.split()[1])

features=[]
for i in data:
    features.append(' '.join(i.text.split()[2:]))

display=[]
for i in data:
    s = text_from_tag(i)
    m = rx_inch.search(s)
    if m:
        # group 1 is bracketed, group 2 is inch-style
        display.append(m.group(1) or m.group(2))
        continue
    m2 = rx_mm.search(s)
    display.append(m2.group(1) + " mm" if m2 else np.nan)

#-------------------

deal=soup.find_all('div',class_="M4DNwV")
deals=[]

for i in deal:
    deals.append(i.text)
    

In [9]:
len(brand),len(model),len(features),len(color),len(display),len(prices),len(MRP),len(offer)

(40, 40, 40, 40, 40, 40, 40, 40)

In [10]:
d={
    'Brand':brand,
    'Model':model,
    'Features':features,
    'Color':color,
    'Display_size':display,
    'belt_size':belt_size,
    'Prices':prices,
    'MRP':MRP,
    'Deal':deals,
    'Offer_percentage':offer}

In [11]:
df=pd.DataFrame(d)

In [12]:
df

,Brand,Model,Features,Color,Display_size,belt_size,Prices,MRP,Deal,Offer_percentage
0,Fire-Boltt,Hurricane,33.02mm (1.3) Curved Glass Display...,Grey Strap,1.3,Free Size,"₹1,499","₹8,999",Bank Offer,83
1,Fire-Boltt,Ninja,Calling Pro Plus 46.5mm (1.83) Display...,Cinnamon Slate Strap,1.83,Free Size,"₹1,049","₹9,999",Super Deals,89
2,Noise,Colorfit,Icon 2 1.8'' Display with Bluetooth Call...,Jet Black Strap,1.8,Regular,"₹1,099","₹5,999",Super Deals,81
3,REPOINT,Big,Smart watch combo 10 in one Smartwatch,Multicolour Strap,10,42,₹757,"₹1,599",Only few left,52
4,Fire-Boltt,Ninja,Calling Pro Plus 46.5mm (1.83) Display...,Pink Strap,1.83,Free Size,₹999,"₹9,999",Super Deals,90
5,GOBOULT,(formerly,"Boult) Crown 1.95"" Screen, BT Calling...",Black Strap,1.95,Free Size,"₹1,099","₹4,499",Super Deals,75
6,Noise,Colorfit,Icon 2 1.8'' Display with Bluetooth Call...,Deep Wine Strap,1.8,Regular,"₹1,099","₹5,999",Super Deals,81
7,Noise,Triumph,"1.85"" Display with swappable dial cases, ...",Grey Strap,1.85,Regular,₹999,"₹8,999",Super Deals,88
8,Fastrack,Revoltt,XR1-1.38|BT Calling|Fast Charge|100+ S...,Beige Strap,NaN,Free Size,"₹1,449","₹3,995",Hot Deal,63
9,Fastrack,XR1-1.38|BT,Calling|Fast Charge|100+ Sports Mo...,Blue Strap,NaN,Free Size,"₹1,449","₹3,995",Hot Deal,63


# Data collection for multiple pages

In [13]:
url1='https://www.flipkart.com/search?q=smart+watches&sid=ajy%2Cbuh&as=on&as-show=on&otracker=AS_QueryStore_HistoryAutoSuggest_1_5_na_na_na&otracker1=AS_QueryStore_HistoryAutoSuggest_1_5_na_na_na&as-pos=1&as-type=HISTORY&suggestionId=smart+watches%7CSmart+Watches&requestId=fe84e6ac-d178-47e4-a3ba-afce1aa1b5b9&as-backfill=on&page='

In [14]:
prices=[]
MRP=[]
offer=[]
color=[]
belt_size=[]
brand=[]
model=[]
features=[]
display = []
deals=[]
# -----------------

def text_from_tag(elem):
    # prefer title attribute (Flipkart uses title), else text
    if hasattr(elem, 'get'):
        return (elem.get('title') or elem.get_text(separator=' ')).strip()
    return str(elem)

rx_inch = re.compile(r'\((\d+(?:\.\d+)?)\)|(\d+(?:\.\d+)?)(?=\s*(?:["\'’]{1,2}|inch\b|in\b))', re.I)
rx_mm   = re.compile(r'(\d+(?:\.\d+)?)\s*mm\b', re.I)

#-------------------------------

for i in range(2,17):
    urls=url1+str(i)
    page=requests.get(urls,headers=headers)
    soup=BeautifulSoup(page.text)

# ----------------

    price=soup.find_all('div',class_="Nx9bqj")
    # prices column
    x=0
    for i in price:
        prices.append(i.text)
        x=x+1
        if x==40:
            break
    mrp=soup.find_all('div',class_="yRaY8j")
    # MRP column
    y=0
    for i in mrp:
        MRP.append(i.text)
        y=y+1
        if y==40:
            break
    off=soup.find_all('div',class_="UkUFwK")
    #offer column
    z=0
    for i in off:
        offer.append(i.text.split('%')[0])
        z=z+1
        if z==40:
            break

# --------------------------

    clr=soup.find_all('div',class_="Br9IW+")
    # color color
    for i in clr:
        color.append(i.text.split(',')[0])
    
    # belt_size column
    for i in clr:
        belt_size.append(i.text.split(',')[1])

# ---------------------------

    data=soup.find_all('a',class_="WKTcLC")
    # brand column
    for i in data:
        brand.append(i.text.split()[0])
    
    # model column
    for i in data:
        model.append(i.text.split()[1])
    
    # featres column
    for i in data:
        features.append(' '.join(i.text.split()[2:]))
    
    # display column
    for i in data:
        s = text_from_tag(i)
        m = rx_inch.search(s)
        if m:
            # group 1 is bracketed, group 2 is inch-style
            display.append(m.group(1) or m.group(2))
            continue
        m2 = rx_mm.search(s)
        display.append(m2.group(1) + " mm" if m2 else np.nan)

#------------------- deals column

    deal=soup.find_all('div',class_="M4DNwV")
    
    for i in deal:
        deals.append(i.text)

In [15]:
len(brand),len(model),len(features),len(color),len(display),len(prices),len(MRP),len(offer)

(600, 600, 600, 600, 600, 600, 596, 600)

In [16]:
len(MRP)

596

In [17]:
from itertools import zip_longest


In [18]:
df1=pd.DataFrame(
    list(zip_longest(brand, model, features, color, display, belt_size, prices, MRP, deals, offer)),
    columns=['Brand', 'Model', 'Features', 'Color', 'Display_size', 'belt_size', 'Prices', 'MRP', 'Deal', 'Offer_percentage']
)

In [19]:
df1

,Brand,Model,Features,Color,Display_size,belt_size,Prices,MRP,Deal,Offer_percentage
0,GOBOULT,(formerly,"Boult) Crown 1.95"" Screen, BT Calling...",Black Strap,1.95,Free Size,"₹1,099","₹4,499",Super Deals,75
1,Fire-Boltt,Ninja,Calling Pro Plus 46.5mm (1.83) Display...,Cinnamon Slate Strap,1.83,Free Size,"₹1,049","₹9,999",Super Deals,89
2,Cellecor,M5,"PLUS 2.01"" IPS HD Display | 700 NITS | BT-C...",Cool Grey Strap,2.01,Free Size,₹999,"₹3,999",Only few left,75
3,Fire-Boltt,Talk,35.3mm (1.39) Bluetooth Calling with Sp...,Deep Blue Strap,1.39,L,"₹1,099","₹9,999",Super Deals,89
4,boAt,Storm,"call 3 w/TBT Navigation,Watch Face Studio,1....",Black Metal Strap,1.83,Free Size,"₹1,198","₹8,499",Super Deals,85
...,...,...,...,...,...,...,...,...,...,...
595,boAt,Storm,"Infinity Plus, 20 Days BIG Battery, ASAP Cha...",Sports White Strap,1.96,Free Size,"₹1,199","₹59,999",Super Deals,90
596,Fire-Boltt,Brillia,Pro 51.3mm (2.02) Largest AMOLED Dis...,Champagne Chain Strap,2.02,Free Size,"₹1,849",None,Super Deals,89
597,Fire-Boltt,Nnja,Talk 35.3mm (1.39) Round Bluetooth Call...,Pink Strap,1.39,"1.39""","₹1,299",None,Bank Offer,45
598,Fastrack,Revoltt,FS2+|2.01'' Display|Functional Crown|B...,Beige Strap,2.01,Free Size,"₹1,628",None,Bank Offer,15


In [20]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Brand             600 non-null    object
 1   Model             600 non-null    object
 2   Features          600 non-null    object
 3   Color             600 non-null    object
 4   Display_size      470 non-null    object
 5   belt_size         600 non-null    object
 6   Prices            600 non-null    object
 7   MRP               596 non-null    object
 8   Deal              600 non-null    object
 9   Offer_percentage  600 non-null    object
dtypes: object(10)
memory usage: 47.0+ KB


# joining multiple dataframes

In [21]:
data=pd.concat([df,df1],ignore_index=True)

In [22]:
data

,Brand,Model,Features,Color,Display_size,belt_size,Prices,MRP,Deal,Offer_percentage
0,Fire-Boltt,Hurricane,33.02mm (1.3) Curved Glass Display...,Grey Strap,1.3,Free Size,"₹1,499","₹8,999",Bank Offer,83
1,Fire-Boltt,Ninja,Calling Pro Plus 46.5mm (1.83) Display...,Cinnamon Slate Strap,1.83,Free Size,"₹1,049","₹9,999",Super Deals,89
2,Noise,Colorfit,Icon 2 1.8'' Display with Bluetooth Call...,Jet Black Strap,1.8,Regular,"₹1,099","₹5,999",Super Deals,81
3,REPOINT,Big,Smart watch combo 10 in one Smartwatch,Multicolour Strap,10,42,₹757,"₹1,599",Only few left,52
4,Fire-Boltt,Ninja,Calling Pro Plus 46.5mm (1.83) Display...,Pink Strap,1.83,Free Size,₹999,"₹9,999",Super Deals,90
...,...,...,...,...,...,...,...,...,...,...
635,boAt,Storm,"Infinity Plus, 20 Days BIG Battery, ASAP Cha...",Sports White Strap,1.96,Free Size,"₹1,199","₹59,999",Super Deals,90
636,Fire-Boltt,Brillia,Pro 51.3mm (2.02) Largest AMOLED Dis...,Champagne Chain Strap,2.02,Free Size,"₹1,849",None,Super Deals,89
637,Fire-Boltt,Nnja,Talk 35.3mm (1.39) Round Bluetooth Call...,Pink Strap,1.39,"1.39""","₹1,299",None,Bank Offer,45
638,Fastrack,Revoltt,FS2+|2.01'' Display|Functional Crown|B...,Beige Strap,2.01,Free Size,"₹1,628",None,Bank Offer,15


### Saving data to csv file

In [36]:
data.to_csv('smart_watch.csv')

## Data cleaning

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640 entries, 0 to 639
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Brand             640 non-null    object
 1   Model             640 non-null    object
 2   Features          640 non-null    object
 3   Color             640 non-null    object
 4   Display_size      505 non-null    object
 5   belt_size         640 non-null    object
 6   Prices            640 non-null    object
 7   MRP               636 non-null    object
 8   Deal              640 non-null    object
 9   Offer_percentage  640 non-null    object
dtypes: object(10)
memory usage: 50.1+ KB


In [25]:
data.shape

(640, 10)

In [26]:
data.isnull().sum()

Brand                 0
Model                 0
Features              0
Color                 0
Display_size        135
belt_size             0
Prices                0
MRP                   4
Deal                  0
Offer_percentage      0
dtype: int64

In [27]:
data['MRP']=data['MRP'].str.replace('₹','').str.replace(',','').astype('float')

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640 entries, 0 to 639
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Brand             640 non-null    object 
 1   Model             640 non-null    object 
 2   Features          640 non-null    object 
 3   Color             640 non-null    object 
 4   Display_size      505 non-null    object 
 5   belt_size         640 non-null    object 
 6   Prices            640 non-null    object 
 7   MRP               636 non-null    float64
 8   Deal              640 non-null    object 
 9   Offer_percentage  640 non-null    object 
dtypes: float64(1), object(9)
memory usage: 50.1+ KB


In [29]:
data.describe()

,MRP
count,636.000000
mean,10808.635220
std,9346.229198
min,1599.000000
25%,5995.000000
50%,8397.000000
75%,11999.000000
max,69999.000000


In [30]:
data['MRP']=data['MRP'].fillna(data['MRP'].median())

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640 entries, 0 to 639
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Brand             640 non-null    object 
 1   Model             640 non-null    object 
 2   Features          640 non-null    object 
 3   Color             640 non-null    object 
 4   Display_size      505 non-null    object 
 5   belt_size         640 non-null    object 
 6   Prices            640 non-null    object 
 7   MRP               640 non-null    float64
 8   Deal              640 non-null    object 
 9   Offer_percentage  640 non-null    object 
dtypes: float64(1), object(9)
memory usage: 50.1+ KB


In [32]:
data['Prices']=data['Prices'].str.replace('₹','').str.replace(',','').astype('float')

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640 entries, 0 to 639
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Brand             640 non-null    object 
 1   Model             640 non-null    object 
 2   Features          640 non-null    object 
 3   Color             640 non-null    object 
 4   Display_size      505 non-null    object 
 5   belt_size         640 non-null    object 
 6   Prices            640 non-null    float64
 7   MRP               640 non-null    float64
 8   Deal              640 non-null    object 
 9   Offer_percentage  640 non-null    object 
dtypes: float64(2), object(8)
memory usage: 50.1+ KB


In [34]:
data['Display_size'].value_counts()

Display_size
1.39     74
1.85     54
1.83     50
1.43     46
2.01     43
1.96     37
2.06     23
1.95     23
1.38     22
1.3      18
2.02     18
1.8      15
1.91      8
1.46      8
1.51      7
46 mm     7
1.97      6
1.32      5
1.69      5
1.81      5
44 mm     4
45 mm     4
40 mm     4
42 mm     3
35 mm     2
1.78      2
2         2
10        1
2.13      1
2.12      1
1.99      1
2.04      1
41 mm     1
1.82      1
1.54      1
49 mm     1
1.52      1
Name: count, dtype: int64

In [35]:
data['Display_size'] = data['Display_size'].astype(str)
